<a href="https://colab.research.google.com/github/marzinouri/persianQA/blob/main/parsbert-finetuned-persianQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
%%capture
!pip install transformers

In [76]:
%%capture
!pip install datasets

# Fine-tuning ParsBERT model on Persian question-answering task

In [3]:
import transformers
model_checkpoint = "HooshvareLab/bert-fa-zwnj-base"
batch_size = 1

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/426k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [26]:
from datasets import load_dataset
datasets = load_dataset("SajjadAyoubi/persian_qa")

In [27]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [28]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [29]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,1925,باشگاه فوتبال بروسیا دورتموند,بروسیا دورتموند یک باشگاه فوتبال آلمانی است که در شهر دورتموند قرار دارد. این باشگاه در بوندس‌لیگا بازی می‌کند و تاکنون ۸ بار قهرمان بوندسلیگا، ۱ بار قهرمان جام در جام اروپا و ۱ بار قهرمانی لیگ قهرمانان اروپا و ۱ بار قهرمانی جام بین قاره‌ای فوتبال را در کارنامه دارد. همچنین این باشگاه پرتماشاگرترین تیم فوتبال در جهان است و همیشه با هواداران خاص خود یاد می‌شود به‌طوری که در هر بازی میانگین ۸۱ هزار نفر در ورزشگاه حضور دارند و این تیم را حمایت می‌کنند، این تیم همراه با بایرن مونیخ دو قدرت اول فوتبال المان به حساب میایند و جدال قدیمی این دو تیم را در کلاسیکر نام می‌برند. بروسیا دورتموند در حال حاضر یکی از موفق‌ترین باشگاه‌های آلمان است و به داشتن ورزشگاهی که بیشترین جایگاه ایستادن تماشاگران در اروپا را دارد، افتخار می‌کند. استادیوم اختصاصی این تیم «وستفالن» با نام تجاری «سیگنال ایدونا پارک» نام دارد که بزرگ‌ترین ورزشگاه آلمان است و بیش از ۸۰ هزار نفر گنجایش دارد و به همین سبب لقب «خانه اپرای فوتبال آلمان» را کسب کرده‌است.,به جز دورتموند چه تیمی جز قدرت های بوندسلیگا است؟,"{'text': ['بایرن مونیخ'], 'answer_start': [471]}"
1,420,دروغ بزرگ,دروغ بزرگ یا دروغ گوبلزی اصطلاح و تکنیکی است در پروپاگاندا، که اولین بار توسط آدولف هیتلر از آن استفاده کرد و نمایندهٔ تکنیکی تبلیغاتی در دنیای سیاست است. آدولف هیتلر در کتاب نبرد من، می‌گوید مردم شکست آلمان در جنگ جهانی اول را به این دلیل پذیرفتند که یهودی‌های دارای نفوذ در مطبوعات از این تکنیک استفاده کردند. از نظر او این روش مستلزم آن است که دروغ چنان عظیم باشد که هیچ‌کس باور نکند که «کسی آنقدر گستاخ باشد که چنین بی‌شرمانه حقیقت را تحریف کند». اولین مورد استفادهٔ دروغ بزرگ در این جملهٔ معروف او مستند شده‌است: «در دروغ بزرگ همواره نیروی قابل باور بودن موجود است.» بعدها یوزف گوبلز که وزیر تبلیغات هیتلر بود این تئوری را با کمی تغییر بدین گونه بیان کرد که دروغ بزرگ یکی از روش‌های تبلیغاتی مورد استفادهٔ انگلستان است که دروغ بزرگی می‌گویند و در ضمن تحت هر شرایطی بر صحت آن پافشاری می‌کنند.شایع است که گوبلز روایت خود از روش دروغ بزرگ را بدون نسبت دادن آن به یهودی‌ها یا متفقین مورد استفاده قرار می‌داد. این روایت بدون منبع شایع‌ترین نمونهٔ استفاده از دروغ بزرگ است و بر اساس آن معمولاً فرض گفته می‌شود که گوبلز خود مبدع تکنیک دروغ بزرگ بوده‌است. در صفحهٔ ۵۱ گزارشی که از طرف دفتر خدمات استراتژیک ایالات متحده در مورد شرح حال روحی هیتلر منتشر شده بود چنین آمده‌است:«اولین قانون او این است که هیچ‌وقت نگذارید مردم دلسرد شوند. مردم دروغ بزرگ را زودتر از دروغ کوچک باور می‌کنند و اگر دروغی را مکرراً تکرار کنید، دیر یا زود آن را باور خواهند کرد.,دروغ کوچک چه مزیتی نسبت به دروغ بزرگ دارد؟,"{'text': [], 'answer_start': []}"
2,4617,خوب، بد، زشت,خوب، بد، زشت یک فیلم درژانر وسترن اسپاگتی حماسی است که توسط سرجو لئونه در سال ۱۹۶۶ در ایتالیا ساخته شد. زبانی که بازیگران این فیلم به آن تکلم می‌کنند مخلوطی از ایتالیایی و انگلیسی است. این فیلم سومین (و آخرین) فیلم از سه‌گانهٔ دلار (Dollars Trilogy) سرجو لئونه است. این فیلم در حال حاضر در فهرست ۲۵۰ فیلم برتر تاریخ سینما در وب‌گاه IMDB با امتیاز ۸٫۸ از ۱۰، رتبهٔ هشتم را به خود اختصاص داده‌است و به عنوان بهترین فیلم وسترن تاریخ سینمای جهان شناخته می‌شود. «خوب» (کلینت ایستوود، در فیلم، با نام «بلوندی») و «زشت» (ایلای والاک، در فیلم، با نام «توکو») با هم کار می‌کنند و با شگرد خاصی، به گول زدن کلانترهای مناطق مختلف و پول درآوردن از این راه می‌پردازند. «بد» (لی وان کلیف) آدمکشی حرفه‌ای است که به‌خاطر پول حاضر به انجام هر کاری است. «بد»، که در فیلم او را «اِنجل آیز (اِینجل آیز)» (به انگلیسی: Angel Eyes) صدا می‌کنند. به‌دنبال گنجی است که در طی جنگ‌های داخلی آمریکا، به دست سربازی به نام «جکسون»، که بعدها به «کارسون» نامش را تغییر داده، مخفی شده‌است.,شخصیت بد در فیلم سرجو لئونه چه هدفی دارد؟,"{'text': ['دنبال گنج'], 'answer_start': [824]}"
3,4123,روان‌کاوی,روان‌کاوی یا روان‌تحلیل‌گری، نظریه‌ای دربارهٔ عملکرد ذهن، اختلال‌های روانی و نام شیوه‌ای روان‌درمانی است که بر این فرض اساسی استوار است که بیشتر فعالیت‌های ذهنی و پردازش آن‌ها در ناخودآگاه رخ می‌دهد. ولی ژاک لاکان می‌گوید روانکاوی کاری است که یک روانکاو می‌کند. واژهٔ «روانکاوی» را اولین بار ابراهیم خواجه‌نوری برساخت و سپس این واژه به همراه بسیاری دیگر از برساخته‌های 

In [30]:
show_random_elements(datasets["validation"])

,id,title,context,question,answers
0,9156,مهاتما گاندی,مهانداس کارامچاند گاندی رهبر سیاسی و معنوی هندی‌ها بود که ملت هند را در راه آزادی از استعمار امپراتوری بریتانیا رهبری کرد. او در طول زندگی‌اش استفاده از هر نوع ترور و خشونت را برای رسیدن به مقاصد رد می‌کرد. فلسفهٔ بی‌خشونتی گاندی که خود نام «ساتیاگراها» روی بسیاری از جنبش‌های مقاومت بدون خشونت در سراسر جهان و تا امروز تأثیر گذاشته‌است. از زمانی که وی مسئولیت رهبری نبرد برای آزادی و کنگره ملی هند در سال ۱۹۱۸ را به عهده گرفت، به عنوان نمادی ملی شناخته شد و میلیون‌ها نفر از مردم او را با لقب مهاتما یا روح بزرگ یاد می‌کردند. هر چند که او از القاب افتخارآمیز بیزار بود ولی امروز هم همگی او را با نام مهاتما گاندی می‌شناسند. سوای اینکه بسیاری او را به عنوان یکی از بزرگ‌ترین رهبران تاریخ تلقی می‌کنند. گاندی توانست با استفاده از شیوهٔ ضد خشونت نافرمانی مدنی استقلال هند را از بریتانیا بگیرد و در نهایت دست امپراتوری بریتانیا را از هند کوتاه کند. اصل ساتیاگراهای گاندی روی بسیاری از فعالان آزادیخواه نظیر دکتر مارتین لوترکینگ، تنزین گیاتسو، لخ والسا، استفان بیکو، آنگ سان سو چی و نلسون ماندلا تأثیر گذاشت.,زمانی که گاندی رهبر مبارزه برای آزادی هند شد مردم به او چه می گفتند؟,"{'text': ['مهاتما یا روح بزرگ', 'مهاتما'], 'answer_start': [494, 494]}"
1,9152,مهاتما گاندی,مهانداس کارامچاند گاندی رهبر سیاسی و معنوی هندی‌ها بود که ملت هند را در راه آزادی از استعمار امپراتوری بریتانیا رهبری کرد. او در طول زندگی‌اش استفاده از هر نوع ترور و خشونت را برای رسیدن به مقاصد رد می‌کرد. فلسفهٔ بی‌خشونتی گاندی که خود نام «ساتیاگراها» روی بسیاری از جنبش‌های مقاومت بدون خشونت در سراسر جهان و تا امروز تأثیر گذاشته‌است. از زمانی که وی مسئولیت رهبری نبرد برای آزادی و کنگره ملی هند در سال ۱۹۱۸ را به عهده گرفت، به عنوان نمادی ملی شناخته شد و میلیون‌ها نفر از مردم او را با لقب مهاتما یا روح بزرگ یاد می‌کردند. هر چند که او از القاب افتخارآمیز بیزار بود ولی امروز هم همگی او را با نام مهاتما گاندی می‌شناسند. سوای اینکه بسیاری او را به عنوان یکی از بزرگ‌ترین رهبران تاریخ تلقی می‌کنند. گاندی توانست با استفاده از شیوهٔ ضد خشونت نافرمانی مدنی استقلال هند را از بریتانیا بگیرد و در نهایت دست امپراتوری بریتانیا را از هند کوتاه کند. اصل ساتیاگراهای گاندی روی بسیاری از فعالان آزادیخواه نظیر دکتر مارتین لوترکینگ، تنزین گیاتسو، لخ والسا، استفان بیکو، آنگ سان سو چی و نلسون ماندلا تأثیر گذاشت.,گاندی چه کسی بود؟,"{'text': ['رهبر سیاسی و معنوی هندی‌ها', 'رهبر سیاسی و معنوی هندی‌ها بود'], 'answer_start': [24, 24]}"
2,9264,لوط,لوط شخصیتی در سفر پیدایش است که در باب ۱۱–۱۴ تا ۱۹ به زندگی او پرداخته شده‌است. او پسر هاران و برادرزادهٔ ابراهیم و ساره بود. در قرآن نیز به لوط اشاره شده‌است. بعد از اینکه هاران مرد، پدربزرگ لوط، تارح، او را به سرپرستی گرفت. مدتی بعد تارح نیز مرد و لوط تحت سرپرستی عمویش ابراهیم قرار گرفت و او را در سفرهایش به کنعان و مصر همراهی کرد. وقتی از مصر بازگشتند، به دلیل کمبود چراگاه میانشان اختلافی پدید آمد و برای جلوگیری از درگیری بین خویشاندان، راهشان از هم جدا شد. لوط به سمت سدوم در دره اردن رفت. در سدوم، زمانی که لوط روی دروازه شهر نشسته بود، دو فرشته الهی برای بررسی گزارش‌های رسیده دربارهٔ گناهان مردم سدوم وارد آن شهر شدند. لوط از آن‌ها پذیرایی کرد و آنان را به خانه خودش برد. در آن زمان، مردان سدوم خانه او را محاصره کردند زیرا قصد تجاوز به مهمانان لوط را داشتند. لوط در عوض به آنان دختران خودش را پیشنهاد کرد اما محاصره‌کنندگان پیشنهادش را نپذیرفتند. آن دو فرشته لوط را به داخل خانه بردند و مردان حاضر در پشت در را کور کردند.,لوط بعد از جدا شدن از ابراهیم کجا رفت؟,"{'text': ['به سمت سدوم در دره اردن', 'سدوم در دره اردن'], 'answer_start': [469, 476]}"
3,9481,تایم,تایم یک مجله خبری هفته‌نامه و روزنامه آنلاین آمریکایی است که در نیویورک مستقر شده و منتشر می‌شود. تایم نخستین بار در ۳ مارس ۱۹۲۳ در شهر نیویورک منتشر شد و سال‌ها توسط بنیانگذار مشترک و تأثیرگذار آن هنری لوس اداره می‌شد. هم‌اکنون سردبیر این مجله ادوارد فلسنتال است. مجله تایم به بررسی رویدادها در زمینه‌های سیاست، اقتصاد، هنر، مذهب، علم، ورزش، تجارت و علوم اجتماعی می‌پردازد. درآمد مجله تایم از طریق آگهی‌های بازرگانی تأمین می‌شود. تایم در بسیاری از کشورهای جهان چاپ می‌شود. ویرایش کانادایی آن به نام Time Canada چاپ تورنتو بود. چاپ این ویرایش از دسامبر ۲۰۰۸ متوقف شد. 

In [31]:
max_length = 384
doc_stride = 128

In [32]:
pad_on_right = tokenizer.padding_side == "right"


In [33]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:

                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [34]:
features = prepare_train_features(datasets['train'][:5])


In [35]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)


## Fine-tuning the model

In [36]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [38]:
torch.cuda.is_available()

True

In [19]:
%%capture
!pip install accelerate -U

In [39]:
model_name = "parsbert-persianQA"
args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
from transformers import default_data_collator

data_collator = default_data_collator

In [41]:
trainer = Trainer(
    model.to(device),
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.846400,2.502019
2,2.188700,2.662051
3,1.282300,3.452055


TrainOutput(global_step=27615, training_loss=2.327479529186541, metrics={'train_runtime': 4235.4144, 'train_samples_per_second': 6.52, 'train_steps_per_second': 6.52, 'total_flos': 5411782452948480.0, 'train_loss': 2.327479529186541, 'epoch': 3.0})

## Evaluation

In [45]:
import torch

for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [46]:
output.start_logits.shape, output.end_logits.shape


(torch.Size([1, 384]), torch.Size([1, 384]))

In [47]:
output.start_logits.argmax(dim=-1), output.end_logits.argmax(dim=-1)


(tensor([19], device='cuda:0'), tensor([19], device='cuda:0'))

In [48]:
n_best_size = 20


In [49]:
import numpy as np

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index:
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": ""
                }
            )

In [50]:
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [51]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names
)

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

In [52]:
raw_predictions = trainer.predict(validation_features)


In [53]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))
max_answer_length = 30


In [78]:
start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
offset_mapping = validation_features[0]["offset_mapping"]
context = datasets["validation"][0]["context"]
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if start_index <= end_index:
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char: end_char]
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[:n_best_size]
valid_answers

[{'score': 17.216526, 'text': 'مادرید'},
 {'score': 15.520025,
  'text': 'باشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 14.956522, 'text': 'در مادرید'},
 {'score': 11.968306,
  'text': 'مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 11.521982, 'text': 'باشگاه فوتبال رئال مادرید'},
 {'score': 11.047681,
  'text': 'رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 7.970262, 'text': 'مادرید'},
 {'score': 7.049637, 'text': 'رئال مادرید'},
 {'score': 5.2895412, 'text': 'مادرید، پایتخت اسپانیا قرار دارد.'},
 {'score': 4.8286414,
  'text': 'یک باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 4.588375, 'text': 'مادرید، پایتخت اسپانیا قرار دارد'},
 {'score': 4.5446954, 'text': 'باشگاه حرفه\u200cای فوتبال است که در مادرید'},
 {'score': 4.1028595, 'text': 'فوتبال است که در مادرید'},
 {'score': 3.5930414,
  'text': 'باشگاه فوتبال رئال مادرید یک باشگاه حرفه\u200cای فوتبال است که در مادرید، پایتخت اسپانیا ق

In [79]:
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [60]:
from tqdm.auto import tqdm

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)
    predictions = collections.OrderedDict()


    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in enumerate(tqdm(examples)):
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []

        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            offset_mapping = features[feature_index]["offset_mapping"]

            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}

        answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
        predictions[example["id"]] = answer

    return predictions

In [61]:
final_predictions = postprocess_qa_predictions(datasets["validation"], validation_features, raw_predictions.predictions)


Post-processing 930 example predictions split into 930 features.


  0%|          | 0/930 [00:00<?, ?it/s]

In [66]:
%%capture
!pip install evaluate

In [68]:
import evaluate

squad_v2 = True
metric = evaluate.load("squad_v2")
formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]

In [71]:
# Step 1: Calculate metrics
metrics = metric.compute(predictions=formatted_predictions, references=references)

# Step 2: Display the metrics
print("Evaluation Metrics:")
for key, value in metrics.items():
    print(f"{key}: {value}")

# Step 3: Show some sample predictions and references
print("\nSample Predictions vs. References:")
for i in range(5):  # Display 5 examples
    prediction = formatted_predictions[i]
    reference = references[i]
    print(f"ID: {prediction['id']}")
    print(f"Prediction: {prediction['prediction_text']}")
    print(f"Reference Answers: {reference['answers']['text']}")
    print("-" * 50)


Evaluation Metrics:
exact: 41.29032258064516
f1: 56.967668492678584
total: 930
HasAns_exact: 31.182795698924732
HasAns_f1: 53.57900414468677
HasAns_total: 651
NoAns_exact: 64.87455197132617
NoAns_f1: 64.87455197132617
NoAns_total: 279
best_exact: 41.39784946236559
best_exact_thresh: 0.0
best_f1: 57.07519537439902
best_f1_thresh: 0.0

Sample Predictions vs. References:
ID: 9101
Prediction: مادرید
Reference Answers: ['مادرید', 'مادرید']
--------------------------------------------------
ID: 9103
Prediction: فیفا
Reference Answers: ['فیفا', 'به انتخاب فیفا']
--------------------------------------------------
ID: 9105
Prediction: ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی
Reference Answers: ['۱۳', '۱۳']
--------------------------------------------------
ID: 9107
Prediction: «سلطنتی»
Reference Answers: ['سلطنتی', 'سلطنتی']
--------------------------------------------------
ID: 9109
Prediction: در مادرید
Reference Answers: ['مادرید، پایتخت اسپانیا', 'اسپانیا']
----------------------------

In [72]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

events.out.tfevents.1728919536.d7f76e625ed3.204.0:   0%|          | 0.00/17.8k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/azizmatin/parsbert-persianQA/commit/fd7451ac05d63847ed1096b71b48ec2ec8f5316a', commit_message='End of training', commit_description='', oid='fd7451ac05d63847ed1096b71b48ec2ec8f5316a', pr_url=None, pr_revision=None, pr_num=None)